## 2.2. Komponenty biblioteki LangChain

### instalacja

In [ ]:
!pip install -q python-dotenv langchain langchain-openai langchain-community langchain-text-splitters faiss-cpu

### konfiguracja .env i modelu

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem. Odpowiadaj zwięźle."),
    ("user", "Streszcz w 1 zdaniu: {tekst}")
])

chain = prompt | llm | StrOutputParser()  # LCEL: prompt → model → parser
wynik = chain.invoke({"tekst": "LangChain ułatwia budowę aplikacji LLM, dostarczając klocki do promptów, pamięci, narzędzi i RAG."})
print(wynik)


### Prompty, Parsowanie, LCEL / Chains
PromptTemplate + LLM + StrOutputParser (LCEL)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem. Odpowiadaj zwięźle."),
    ("user", "Streszcz w 1 zdaniu: {tekst}")
])

chain = prompt | llm | StrOutputParser()  # LCEL: prompt → model → parser
wynik = chain.invoke({"tekst": "LangChain ułatwia budowę aplikacji LLM, dostarczając klocki do promptów, pamięci, narzędzi i RAG."})
print(wynik)


### Tools (narzędzia, które może wywołać model)
prosty tool (kalkulator) + agent ReAct

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: float, b: float) -> float:
    """Zwraca sumę a+b."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Zwraca iloczyn a*b."""
    return a * b

tools = [add, multiply]

from langchain import agents
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

react_prompt = PromptTemplate.from_template(
"""Jesteś pomocnym asystentem. Masz dostęp do narzędzi.
Używaj ich tylko gdy potrzebne. Odpowiadaj po polsku.

Pytanie: {input}
{agent_scratchpad}"""
)

agent = create_react_agent(llm, tools, react_prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

odp = executor.invoke({"input": "Policz (12 + 7) * 3 i podaj wynik."})
print("\nWynik końcowy:", odp["output"])


### budowa VectorStore (FAISS) i Retrievera

In [ ]:
from langchain_community.vectorstores import FAISS

emb = OpenAIEmbeddings()  # wymaga OPENAI_API_KEY
vectorstore = FAISS.from_texts(splitted, embedding=emb)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

query = "Po co używa się retrievera?"
context = retriever.get_relevant_documents(query)
print("Znaleziony kontekst:")
for i, c in enumerate(context, 1):
    print(f"{i}.", c.page_content)


### prosty łańcuch RAG (prompt + kontekst + LLM)

In [ ]:
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Udziel precyzyjnej odpowiedzi wyłącznie na podstawie KONTEKSTU. Jeśli brak danych — powiedz, że nie wiesz."),
    ("system", "KONTEKST:\n{context}"),
    ("user", "{question}")
])

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Czym jest FAISS i do czego służy?"))


### Memory (historia rozmowy / stan)
RunnableWithMessageHistory (pamięć czatu)

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Prowadzisz przyjazną rozmowę i pamiętasz kontekst."),
    ("user", "{input}")
])

chat_chain = chat_prompt | llm | StrOutputParser()

store = {}  # prosta “baza” historii po session_id

def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_memory = RunnableWithMessageHistory(
    chat_chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history",
)

sid = "demo-session-1"
print(chain_with_memory.invoke({"input": "Cześć! Mam na imię Michał."},
                               config={"configurable": {"session_id": sid}}))
print(chain_with_memory.invoke({"input": "Jak mam na imię?"},
                               config={"configurable": {"session_id": sid}}))


### LangSmith (tracing) — opcjonalnie
włącz śledzenie (jeśli masz konto)

In [ ]:
# Opcjonalnie (wymaga konta):
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = "<TWÓJ_KLUCZ>"
# os.environ["LANGSMITH_PROJECT"] = "kurs-demo"
print("LangSmith: ustaw zmienne środowiskowe, aby włączyć tracing.")


### LangGraph (mini-graf stanów)
minimalny graf: plan → odpowiedź

In [ ]:
# Minimalny przykład z LangGraph: dwa węzły (plan, odpowiedź)
!pip -q install langgraph

from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    question: str
    plan: str
    answer: str

def plan_node(state: State):
    q = state["question"]
    return {"plan": f"1) Zrozumieć pytanie: {q}\n2) Odpowiedzieć krótko."}

def answer_node(state: State):
    p = state["plan"]
    msg = f"Wykonuję plan:\n{p}\n\nOdpowiedź: To prosty przykład grafu stanów w LangGraph."
    return {"answer": msg}

g = StateGraph(State)
g.add_node("plan", plan_node)
g.add_node("answer", answer_node)
g.set_entry_point("plan")
g.add_edge("plan", "answer")
g.add_edge("answer", END)
app = g.compile()

out = app.invoke({"question": "Co to jest LangGraph?"})
print(out["answer"])
